# MetaDSL + UArray

In this notebook, Hameer and I (Saul) try out some ways to have MetaDSL and UArray place nice together.

First, let's installed the latest uarray

In [1]:
!pip install -U git+https://github.com/Quansight-Labs/uarray.git

  Cloning https://github.com/Quansight-Labs/uarray.git to /private/var/folders/m7/t8dvwtnn32z84333p845tly40000gn/T/pip-req-build-vv251_40
  Stored in directory: /private/var/folders/m7/t8dvwtnn32z84333p845tly40000gn/T/pip-ephem-wheel-cache-6m7r4pvj/wheels/3d/1a/bf/60f787ed8f0ac071de28d869eb644793856923ac4688c14544
Successfully built uarray
  Found existing installation: uarray 0.4+168.g345664c
    Uninstalling uarray-0.4+168.g345664c:
      Successfully uninstalled uarray-0.4+168.g345664c


Now, there are two ways of integrating:

1. `metadsl -> uarray` Build metadsl graph of numpy expression using `metadsl.nonumpy.compat` and then execute them using a uarray backend
2. `uarray -> metadsl` Build up a metadsl graph using `unumpy`, by defining a `metadsl` backend for `uarray`.

Both can make sense, so let's do a simple implementation of both sides and see how we can use them togther. We will start with (2), adding a `metadsl` backend for `uarray`: 

In [1]:
import uarray
import unumpy
import typing
import unumpy.multimethods

In [2]:
import metadsl

In [3]:
metadsl_backend = uarray.Backend()
uarray.register_backend(metadsl_backend)

Now let's just register one call, the `arange` call, and dispatch this by creating a call graph

In [5]:
class Array(metadsl.Instance):
    pass


@metadsl.call(lambda start, stop, stride: Array)
def arange(start: int, stop: int, stride: int) -> Array:
    ...

@metadsl.call(lambda shape: Array)
def zeros(shape: typing.Tuple[int]) -> Array:
    ...

@metadsl.call(lambda left, right: Array)
def add(left: Array, right: Array) -> Array:
    ...

@metadsl.call(lambda a: Array)
def sum(a: Array) -> Array:
    ...
    
uarray.register_implementation(unumpy.arange, metadsl_backend)(arange)
uarray.register_implementation(unumpy.zeros, metadsl_backend)(zeros)
uarray.register_implementation(unumpy.add, metadsl_backend)(add)
uarray.register_implementation(unumpy.sum, metadsl_backend)(sum)


<function __main__.sum(a: __main__.Array) -> __main__.Array>

Now we can use the metadsl backend to create a graph

In [6]:
with uarray.set_backend(metadsl_backend):
    left = unumpy.arange(0, 10, 2)
    right = unumpy.sum(unumpy.zeros(10))
    together = unumpy.add(left, right)

TypeError: inner_inner() got an unexpected keyword argument 'out'

And here is a simple versio of (1) which takes a metadsl graph of unumpy expressions and executes them:

In [13]:
import unumpy.numpy_backend

def execute_graph(x):
    """
    Test version of executing a graph, in the form of a single funtion with primitive args.
    """
    val = x.__value__
    fn = val.function
    args = val.args
    return fn(*(a.__value__ for a in args))

In [14]:
print(execute_graph(aranged_call_wrapped(1, 10, 2)))

[1 3 5 7 9]


Now we can use them together!

First we make a graph, using unumpy expression, then we execute that graph:

In [15]:
with uarray.set_backend(MetaDSLBackend, coerce=True):

    x = unumpy.arange(0, 10, 2)
    print(x)
    with uarray.skip_backend(MetaDSLBackend):
        print(execute_graph(x))

Instance(__value__=Call(function=<function arange at 0x106e91730>, args=(Instance(__value__=0), Instance(__value__=10), Instance(__value__=2))))
[0 2 4 6 8]
